In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import logging
import sys
import time, datetime
from typing import Dict, Iterable
import os
import gzip
import json

from google.colab import drive

# Mount Google Drive to the current Colab session
drive.mount('/content/drive')

# Verify that your drive is accessible
!ls /content/drive/MyDrive

Mounted at /content/drive
'16 б.gdoc'			  deleteme.zip		   share
'6 фото.     5 лет блондин.JPG'   futuristic-office.jpeg  'Tanya Passport.JPG'
'Colab Notebooks'		  rental_houses.gsheet	  'лечение травами 3.gdoc'


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
def remove_triple_backticks_at_edges(text: str) -> str:
    """
    Removes triple backticks (` ``` `) if they appear at the beginning or the end of the text.

    :param text: The input text as a string.
    :return: A string with triple backticks removed from the edges.
    """
    s1 = "```python\n"
    s2 = "```python"
    s3 = "```"
    for s in [s1, s2, s3]:
        if text.startswith(s):
            text = text[len(s):].lstrip()      # Remove a version of backticks + extra whitespace at the start
    if text.endswith(s3):
        text = text[: -len(s3)].rstrip()       # Remove the backticks + extra whitespace at the end
    return text


def add_import(text: str) -> str:
    ''' Adds import statements stripped by LLM '''
    text = text.strip()
    s = 'from typing import List'
    if s not in text:
        text = s + '\n\n' + text
    return text

In [ ]:
def write_jsonl(filename: str, data: Iterable[Dict], append: bool = False):
    """
    Writes an iterable of dictionaries to jsonl
    """
    if append:
        mode = 'ab'
    else:
        mode = 'wb'
    filename = os.path.expanduser(filename)
    if filename.endswith(".gz"):
        with open(filename, mode) as fp:
            with gzip.GzipFile(fileobj=fp, mode='wb') as gzfp:
                for x in data:
                    gzfp.write((json.dumps(x) + "\n").encode('utf-8'))
    else:
        with open(filename, mode) as fp:
            for x in data:
                fp.write((json.dumps(x) + "\n").encode('utf-8'))


def stream_jsonl(filename: str) -> Iterable[Dict]:
    """
    Parses each jsonl line and yields it as a dictionary
    """
    if filename.endswith(".gz"):
        with open(filename, "rb") as gzfp:
            with gzip.open(gzfp, 'rt') as fp:
                for line in fp:
                    if any(not x.isspace() for x in line):
                        yield json.loads(line)
    else:
        with open(filename, "r") as fp:
            for line in fp:
                if any(not x.isspace() for x in line):
                    yield json.loads(line)


def read_problems(evalset_file: str) -> Dict[str, Dict]:
    return {task["task_id"]: task for task in stream_jsonl(evalset_file)}

In [ ]:
# read dataset
file = '/content/drive/My Drive/Colab Notebooks/data/MBPP_Test.jsonl.gz'
tasks = read_problems(file)
print(f'Type of tasks: {type(tasks)}\n')

counter = 0
for k,v in tasks.items():
    if counter == 10:
        break
    print(f'task_id: {k}', type(k))
    for k2, v2 in v.items():
        print(f'\n{k2}:\n{v2}')
    print('\n' + '='*100 + '\n')
    counter += 1

Type of tasks: <class 'dict'>

task_id: 11 <class 'str'>

task_id:
11

test:

assert remove_Occ("hello","l") == "heo"
assert remove_Occ("abcda","a") == "bcd"
assert remove_Occ("PHP","P") == "H"

prompt:
Write a python function to remove first and last occurrence of a given character from the string.

canonical_solution:
def remove_Occ(s,ch): 
    for i in range(len(s)): 
        if (s[i] == ch): 
            s = s[0 : i] + s[i + 1:] 
            break
    for i in range(len(s) - 1,-1,-1):  
        if (s[i] == ch): 
            s = s[0 : i] + s[i + 1:] 
            break
    return s 

entry_point:
None


task_id: 12 <class 'str'>

task_id:
12

test:

assert sort_matrix([[1, 2, 3], [2, 4, 5], [1, 1, 1]])==[[1, 1, 1], [1, 2, 3], [2, 4, 5]]
assert sort_matrix([[1, 2, 3], [-2, 4, -5], [1, -1, 1]])==[[-2, 4, -5], [1, -1, 1], [1, 2, 3]]
assert sort_matrix([[5,8,9],[6,4,3],[2,1,4]])==[[2, 1, 4], [6, 4, 3], [5, 8, 9]]

prompt:
Write a function to sort a given matrix in ascending order accordi

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "deepseek-ai/deepseek-coder-6.7b-instruct",
    trust_remote_code=True,
)
model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/deepseek-coder-6.7b-instruct",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16).cuda()
print(f'Model device: {model.device}')

tokenizer_config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

Model device: cuda:0


In [ ]:
print(f'Model temperature: {model.config.temperature}. Model top_p: {model.config.top_p}')

In [ ]:
def generate_response(prompt):
  messages=[
      { 'role': 'user', 'content': prompt }
  ]
  inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
  outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
  res = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
  return res

In [ ]:
task = tasks['15']
prompt_prefix = 'You are an expert Python programmer, and here is your task: {} Do not output any clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:\n'

print(prompt_prefix.format(task['prompt']) + task['test'])

You are an expert Python programmer, and here is your task: Write a function to split a string at lowercase letters. Do not output any clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert split_lowerstring("AbCd")==['bC','d']
assert split_lowerstring("Python")==['y', 't', 'h', 'o', 'n']
assert split_lowerstring("Programming")==['r', 'o', 'g', 'r', 'a', 'm', 'm', 'i', 'n', 'g']


In [ ]:
# to save results
model_nickname = 'deepseek-coder-6.7b-instruct'
time_stamp     = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-2]
file_name      = f'/content/drive/My Drive/Colab Notebooks/logs/{model_nickname}_samples_{time_stamp}.jsonl'
file_name

'/content/drive/My Drive/Colab Notebooks/logs/deepseek-coder-6.7b-instruct_samples_20250105_100053_5217.jsonl'

In [ ]:
num_samples_per_task = 1
completions = []

for task_id, task_body in tasks.items():
    print(task_id)
    #if task_id == '15':
    #        break
    for i in range(num_samples_per_task):
        start_time  = time.time()
        full_prompt = prompt_prefix.format(task_body['prompt']) + task_body['test']
        try:
            completion  = generate_response( full_prompt )
        except Exception as e:
            completion = f"Error generating a completion:\n{e}"

        completion = remove_triple_backticks_at_edges(completion.strip())
        completions.append( {'task_id': task_id, 'completion': completion} )
        write_jsonl(file_name, completions)
        print('Prompt:\n', full_prompt, '\n', '-'*75, '\n', sep='')
        print('Completion:\n', completion, sep='')
        print(f"\nTime elapsed: {(time.time() - start_time):.4f} seconds\n", '\n', '='*75, '\n', sep='')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


11


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Prompt:
You are an expert Python programmer, and here is your task: Write a python function to remove first and last occurrence of a given character from the string. Do not output any clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert remove_Occ("hello","l") == "heo"
assert remove_Occ("abcda","a") == "bcd"
assert remove_Occ("PHP","P") == "H"
---------------------------------------------------------------------------

Completion:
Sure, here is the Python function that removes the first and last occurrence of a given character from a string:

```python
def remove_Occ(s, ch):
    first_index = s.find(ch)
    if first_index != -1:
        last_index = s.rfind(ch)
        return s[:first_index] + s[first_index+1:last_index] + s[last_index+1:]
    else:
        return s

# Test cases
assert remove_Occ("hello","l") == "heo"
assert remove_Occ("abcda","a") == "bcd"
assert remove_Occ("PHP","P") == "H"
```

This function first finds th

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


...

Prompt:
You are an expert Python programmer, and here is your task: Write a function to remove specific words from a given list. Do not output any clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert remove_words(['red', 'green', 'blue', 'white', 'black', 'orange'],['white', 'orange'])==['red', 'green', 'blue', 'black']
assert remove_words(['red', 'green', 'blue', 'white', 'black', 'orange'],['black', 'orange'])==['red', 'green', 'blue', 'white']
assert remove_words(['red', 'green', 'blue', 'white', 'black', 'orange'],['blue', 'white'])==['red', 'green', 'black', 'orange']
---------------------------------------------------------------------------

Completion:
Sure, here is the Python function that removes specific words from a given list:

```python
def remove_words(lst, words_to_remove):
    return [word for word in lst if word not in words_to_remove]
```

You can use this function as follows:

```python
print(remove_words([

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Prompt:
You are an expert Python programmer, and here is your task: Write a function to check if the common elements between two given lists are in the same order or not. Do not output any clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert same_order(["red","green","black","orange"],["red","pink","green","white","black"])==True
assert same_order(["red","pink","green","white","black"],["white","orange","pink","black"])==False
assert same_order(["red","green","black","orange"],["red","pink","green","white","black"])==True
---------------------------------------------------------------------------

Completion:
Here is the Python function to solve the problem:

```python
def same_order(list1, list2):
    common_elements = [value for value in list1 if value in list2]
    return common_elements == list1
```

You can run the tests with the following code:

```python
assert same_order(["red","green","black","orange"],["red","pink","

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Prompt:
You are an expert Python programmer, and here is your task: Write a python function to find the average of odd numbers till a given odd number. Do not output any clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert average_Odd(9) == 5
assert average_Odd(5) == 3
assert average_Odd(11) == 6
---------------------------------------------------------------------------

Completion:
Sure, here is the Python function that calculates the average of odd numbers till a given odd number:

```python
def average_Odd(n):
    sum_of_odd_numbers = sum(range(1, n + 1, 2))
    count_of_odd_numbers = (n + 1) // 2
    return sum_of_odd_numbers // count_of_odd_numbers

# Test cases
assert average_Odd(9) == 5
assert average_Odd(5) == 3
assert average_Odd(11) == 6
```

This function works by using the `range` function to generate a sequence of odd numbers from 1 to `n` (inclusive). The `sum` function is then used to add up these numbers, and 